In [1]:
import pickle
import pandas as pd
import os           
import numpy as np
import re
from scipy.stats import pearsonr
import itertools
import plotly.express as px
#import brightway2 as bw 
import random 
import string
import sys

In [2]:
sys.path.append('../../Module')  #two levels up & then down to Module folder
from common_mod import *
from mod_resIC import *

In [3]:
lcia_name = "AWARE"
pairwise_name = ["SPvsopenLCA", "SPvsWULCA",  "openLCAvsWULCA"]

<h3>RAW AWARE CFs from SP_v9 & openLCA </h3>

In [4]:
olca_aware = pd.read_excel('../../Data_rawCFs/AWARE_CF_openLCA.xlsx' )
print(len(olca_aware))
olca_aware.head()

2270


,Flow,Category,Flow property,Factor,Unit,Uncertainty
0,Cooling water,Emission to water/ocean,Volume,0.00000,m3/m3,none
1,Cooling water,Emission to water/unspecified,Volume,-42.95000,m3/m3,none
2,Fresh water (obsolete),Emission to water/surface water,Volume,-42.95000,m3/m3,none
3,Water,Emission to water/ground water,Volume,-42.95000,m3/m3,none
4,Water,Emission to water/ground water,Mass,-0.04295,m3/kg,none


In [5]:
SP9_aware = pd.read_excel('../../Data_rawCFs/SP_9_AWARE.xlsx' )
print(len(SP9_aware))
SP9_aware.head()

1807


,Compartment,Subcompartment,Substance,CAS,Factor,Unit
0,Raw,(unspecified),"Water, cooling, drinking",007732-18-5,0.04295,m3 / kg
1,Raw,(unspecified),"Water, cooling, surface",007732-18-5,0.04295,m3 / kg
2,Raw,(unspecified),"Water, cooling, unspecified natural origin, AD",007732-18-5,74.67000,m3 / m3
3,Raw,(unspecified),"Water, cooling, unspecified natural origin, AE",007732-18-5,18.56000,m3 / m3
4,Raw,(unspecified),"Water, cooling, unspecified natural origin, AF",007732-18-5,57.20000,m3 / m3


<h3>RAW AWARE CFs from WULCA </h3>
source: https://wulca-waterlca.org/aware/download-aware-factors/

CSV file, deleted Intro headers, read-in & cleanup

In [6]:
WULCA_aware = pd.read_csv('../../Data_rawCFs/Aware102_SimaPro8_4_NOheader.csv', header=None)
print(len(WULCA_aware))
#WULCA_aware[0:29] #first 30 rows are for global, messed up data, from row index 29, for each country 
WULCA_aware29 = WULCA_aware[29:]

5449


<h4> WULCA contains agri, non-agri, and generic CFs, for pairwise comp. with SP_v9 and openLCA CFs, select only generic CFs</h4>

In [7]:
def select_df(rawdf, containstring):
    sliced_df =  rawdf[rawdf[0].str.contains(containstring, na=False)].append(
                 rawdf[rawdf[1].str.contains(containstring, na=False)]).append( 
                 rawdf[rawdf[2].str.contains(containstring, na=False)]).append( 
                 rawdf[rawdf[3].str.contains(containstring, na=False)]).append(
                 rawdf[rawdf[4].str.contains(containstring, na=False)])
    return sliced_df

In [8]:
wulca_agri_nonagri =  select_df(WULCA_aware,"agri") 
WULCA_aware_new = WULCA_aware.drop(wulca_agri_nonagri.index)

In [9]:
WULCA_aware_new.head()

,0,1,2,3,4
0,Water;(unspecified);Water;007732-18-5;-0.04295...,NaN,NaN,NaN,NaN
1,Water;ocean;Water;007732-18-5;0;kg,NaN,NaN,NaN,NaN
2,Raw;(unspecified);Water,cooling,unspecified natural origin/m3;007732-18-5;20....,NaN,NaN
3,Raw;(unspecified);Water,lake;007732-18-5;42.95353087;m3,NaN,NaN,NaN
4,Raw;(unspecified);Water,river;007732-18-5;42.95353087;m3,NaN,NaN,NaN


In [10]:
# split each cell by ";" and save to Coln_split_i variable 
for col in WULCA_aware_new.columns:
    for i in range(4): 
        locals()["Col"+str(col)+"_split"+str(i)] = WULCA_aware_new[col].str.split(";").str[i]

In [11]:
Col0_split0.values

array(['Water', 'Water', 'Raw', ..., 'Water', '', 'End'], dtype=object)

In [12]:
# the first splitted string contains flow names from raw CSV data:
flow_fullname = pd.Series([Col0_split0.values, Col1_split0.values, Col2_split0.values, Col3_split0.values])
newname = []
for i in range(len(flow_fullname[0])): 
    newname.append(flow_fullname[0][i] + ", " + str(flow_fullname[1][i]) + ", " +
                   str(flow_fullname[2][i]) + ", " + str(flow_fullname[3][i]))

In [13]:
newname2 = []
for x in newname: 
    x = (x.replace(" nan,", "" ))
    x = x.replace(", nan", "")
    x = x.replace(", ", ",") 
    x = x.replace("Raw, ", "Water, ")
    newname2.append(x)
    #print(xx)

In [14]:
WULCA_aware_new["newname"] = newname2

In [15]:
# the third splitted string (_split2) contains CF values :
flow_CF = pd.Series([Col0_split2.values, Col1_split2.values, Col2_split2.values, Col3_split2.values])

newCF = []
for i in range(len(flow_CF[0])): 
    newCF.append(str(flow_CF[0][i]) + ", " + str(flow_CF[1][i]) + ", " +
                   str(flow_CF[2][i]) + ", " + str(flow_CF[3][i]))

#newCF

In [16]:
newCF2 = []
for x in newCF: 
    x = x.replace("Water,", "" )
    x = x.replace(", nan, nan", "")
    x = x.replace("Water/m3, nan", "")
    x=x.replace("Water/m3, ", "")
    x = x.replace(", nan", "")
    x = x.replace("nan, ", "")
    newCF2.append(x)
    #print(x)

In [17]:
WULCA_aware_new["Factor"] = newCF2
#WULCA_aware_new["Factor"] = WULCA_aware_new["Factor"].astype(float)
try:
    WULCA_aware_new["Factor"]  = WULCA_aware_new["Factor"].astype(float)
except: 
    ValueError

In [18]:
WULCA_aware_new.head() #only the newly added "newname" and "factor" cols will be used for pairwise comp. 

,0,1,2,3,4,newname,Factor
0,Water;(unspecified);Water;007732-18-5;-0.04295...,NaN,NaN,NaN,NaN,Water,nan
1,Water;ocean;Water;007732-18-5;0;kg,NaN,NaN,NaN,NaN,Water,nan
2,Raw;(unspecified);Water,cooling,unspecified natural origin/m3;007732-18-5;20....,NaN,NaN,"Water, cooling, unspecified natural origin/m3",20.30408632
3,Raw;(unspecified);Water,lake;007732-18-5;42.95353087;m3,NaN,NaN,NaN,"Water, lake",42.95353087
4,Raw;(unspecified);Water,river;007732-18-5;42.95353087;m3,NaN,NaN,NaN,"Water, river",42.95353087


<h3> pairwise comp. between WULCA, SP, openLCA CFs , comp. by flow names </h3>

In [19]:
# create 3 empty dict: dict_SPvsWULCA; dict_SPvsopenLCA; dict_openLCAvsWULCA
for name in pairwise_name:
    #print(name)
    keys = [name+"_commonEF_sumtable", name+"_commonEF_list", name + "_result_diffEFs", name + "_result_corr" ]
    globals()['dict_%s' % name] = dict.fromkeys(keys, None)

In [20]:
k_list = list(dict_SPvsWULCA)
dict_SPvsWULCA[k_list[0]],dict_SPvsWULCA[k_list[1]] = res_compare_included_EF(SP9_aware, "Substance",   WULCA_aware_new, "newname",  to_print = "no" )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 1314


In [21]:
dict_SPvsWULCA[k_list[2]] = pd.DataFrame (
        {'SP_value': final_EF_combined(SP9_aware, "Substance", "Factor", WULCA_aware_new, "newname", "Factor", dict_SPvsWULCA[k_list[1]])[0] , 
         'WULCA_value': final_EF_combined(SP9_aware, "Substance", "Factor", WULCA_aware_new, "newname", "Factor", dict_SPvsWULCA[k_list[1]])[1] },
        index = dict_SPvsWULCA[k_list[1]]
    )

Different value for a same flow: Water, MG
Different value for a same flow: Water, RME
Different value for a same flow: Water, AD
Different value for a same flow: Water, DK
Different value for a same flow: Water, TO
Different value for a same flow: Water, AW
Different value for a same flow: Water, KM
Different value for a same flow: Water, RoW
Different value for a same flow: Water, NC
Different value for a same flow: Water, AT
Different value for a same flow: Water, SD
Different value for a same flow: Water, GR
Different value for a same flow: Water, CZ
Different value for a same flow: Water, BF
Different value for a same flow: Water, TJ
Different value for a same flow: Water, CN
Different value for a same flow: Water, GM
Different value for a same flow: Water, NA
Different value for a same flow: Water, CL
Different value for a same flow: Water, BB
Different value for a same flow: Water, IS
Different value for a same flow: Water, UA
Different value for a same flow: Water, ER
Different

Different value for a same flow: Water, AE
Different value for a same flow: Water, Europe without Switzerland
Different value for a same flow: Water, RAF
Different value for a same flow: Water, AO
Different value for a same flow: Water, LV
Different value for a same flow: Water, LS
Different value for a same flow: Water, SZ
Different value for a same flow: Water, IM
Different value for a same flow: Water, SO
Different value for a same flow: Water, TZ
Different value for a same flow: Water, LY
Different value for a same flow: Water, IR
Different value for a same flow: Water, CH
Different value for a same flow: Water, MY
Different value for a same flow: Water, IAI Area, Russia & RER w/o EU27 & EFTA
Different value for a same flow: Water, VG
Different value for a same flow: Water, NI
Different value for a same flow: Water, IE
Different value for a same flow: Water, MX
Different value for a same flow: Water, JP
Different value for a same flow: Water, VE
Different value for a same flow: Wat

Different value for a same flow: Water, PH
Different value for a same flow: Water, BH
Different value for a same flow: Water, VU
Different value for a same flow: Water, EG
Different value for a same flow: Water, HU
Different value for a same flow: Water, CD
Different value for a same flow: Water, OCE
Different value for a same flow: Water, MT
Different value for a same flow: Water, SV
Different value for a same flow: Water, PT
Different value for a same flow: Water, TG
Different value for a same flow: Water, CU
Different value for a same flow: Water, IT
Different value for a same flow: Water, MD
Different value for a same flow: Water, CA
Different value for a same flow: Water, PM
Different value for a same flow: Water, TM
Different value for a same flow: Water, NP
Different value for a same flow: Water, AE
Different value for a same flow: Water, Europe without Switzerland
Different value for a same flow: Water, RAF
Different value for a same flow: Water, AO
Different value for a same f

Different value for a same flow: Water, DZ
Different value for a same flow: Water
Different value for a same flow: Water, CR
Different value for a same flow: Water, LT
Different value for a same flow: Water, GQ
Different value for a same flow: Water, GB
Different value for a same flow: Water, IAI Area, South America
Different value for a same flow: Water, LR
Different value for a same flow: Water, CO
Different value for a same flow: Water, GLO
Different value for a same flow: Water, EH
Different value for a same flow: Water, PH
Different value for a same flow: Water, BH
Different value for a same flow: Water, VU
Different value for a same flow: Water, EG
Different value for a same flow: Water, HU
Different value for a same flow: Water, CD
Different value for a same flow: Water, OCE
Different value for a same flow: Water, IAI Area, Gulf Cooperation Council
Different value for a same flow: Water, MT
Different value for a same flow: Water, SV
Different value for a same flow: Water, PT
Dif

Different value for a same flow: Water, RO
Different value for a same flow: Water, GA
Different value for a same flow: Water, BW
Different value for a same flow: Water, DM
Different value for a same flow: Water, LU
Different value for a same flow: Water, US
Different value for a same flow: Water, MQ
Different value for a same flow: Water, NG
Different value for a same flow: Water, IQ
Different value for a same flow: Water, ES
Different value for a same flow: Water, DZ
Different value for a same flow: Water
Different value for a same flow: Water, CR
Different value for a same flow: Water, LT
Different value for a same flow: Water, GQ
Different value for a same flow: Water, GB
Different value for a same flow: Water, LR
Different value for a same flow: Water, CO
Different value for a same flow: Water, GLO
Different value for a same flow: Water, EH
Different value for a same flow: Water, PH
Different value for a same flow: Water, BH
Different value for a same flow: Water, VU
Different valu

In [22]:
k_list = list(dict_SPvsopenLCA)
dict_SPvsopenLCA[k_list[0]],dict_SPvsopenLCA[k_list[1]] = res_compare_included_EF(SP9_aware, "Substance", olca_aware,  "Flow", to_print = "yes" )

EF in A but not in B: ['Water, surface water consumption']
EF in B but not in A: ['Water, process', 'Water, cooling, unspecified natural origin', 'Water, well, unspecified', 'Water (fresh water)', 'Water, surface', 'Water, process, unspecified natural origin', 'Fresh water (obsolete)', 'Water, unspecified natural origin', 'Water (river water from technosphere cooling water)', 'Water (lake water from technosphere, cooling water)', 'Water, process and cooling, surface', 'Water, cooling, well, in ground', 'Water (river water from technosphere turbined)', 'Water/kg', 'Water, ground', 'Water, Surface water consumption', 'Water, unspecified origin', 'Water Cooling fresh']
N of common EFs to be used is: 1574


similar to recipe_water consumption, for AWARE,  diff. CF value for the same EF (becaz diff. type (emission/resource) and diff. compartment: ocean = 0, freshwater = 1), for simplicity, they will be assigned nan value for corr. matrix 

In [23]:
dict_SPvsopenLCA[k_list[2]] = pd.DataFrame (
        {'SP_value': final_EF_combined(SP9_aware, "Substance", "Factor", olca_aware, "Flow", "Factor", dict_SPvsopenLCA[k_list[1]])[0] , 
         'olca_value': final_EF_combined(SP9_aware, "Substance", "Factor", olca_aware, "Flow", "Factor", dict_SPvsopenLCA[k_list[1]])[1] },
        index = dict_SPvsopenLCA[k_list[1]]
    )

Different value for a same flow: Water, MG
Different value for a same flow: Water, RME
Different value for a same flow: Water, AD
Different value for a same flow: Water, DK
Different value for a same flow: Water, TO
Different value for a same flow: Water, AW
Different value for a same flow: Water, KM
Different value for a same flow: Water, RoW
Different value for a same flow: Water, NC
Different value for a same flow: Water, AT
Different value for a same flow: Water, SD
Different value for a same flow: Water, GR
Different value for a same flow: Water, CZ
Different value for a same flow: Water, BF
Different value for a same flow: Water, TJ
Different value for a same flow: Water, CN
Different value for a same flow: Water, GM
Different value for a same flow: Water, NA
Different value for a same flow: Water, CL
Different value for a same flow: Water, BB
Different value for a same flow: Water, IS
Different value for a same flow: Water, UA
Different value for a same flow: Water, ER
Different

Different value for a same flow: Water, SO
Different value for a same flow: Water, TZ
Different value for a same flow: Water, LY
Different value for a same flow: Water, IR
Different value for a same flow: Water, CH
Different value for a same flow: Water, MY
Different value for a same flow: Water, IAI Area, Russia & RER w/o EU27 & EFTA
Different value for a same flow: Water, VG
Different value for a same flow: Water, NI
Different value for a same flow: Water, IE
Different value for a same flow: Water, Europe, without Russia and Turkey
Different value for a same flow: Water, MX
Different value for a same flow: Water, JP
Different value for a same flow: Water, VE
Different value for a same flow: Water, GW
Different value for a same flow: Water, LC
Different value for a same flow: Water, RER
Different value for a same flow: Water, KR
Different value for a same flow: Water, KN
Different value for a same flow: Water, AG
Different value for a same flow: Water, TD
Different value for a same fl

Different value for a same flow: Water, IAI Area, South America
Different value for a same flow: Water, LR
Different value for a same flow: Water, CO
Different value for a same flow: Water, GLO
Different value for a same flow: Water, EH
Different value for a same flow: Water, PH
Different value for a same flow: Water, BH
Different value for a same flow: Water, VU
Different value for a same flow: Water, EG
Different value for a same flow: Water, HU
Different value for a same flow: Water, CD
Different value for a same flow: Water, OCE
Different value for a same flow: Water, IAI Area, Gulf Cooperation Council
Different value for a same flow: Water, MT
Different value for a same flow: Water, SV
Different value for a same flow: Water, PT
Different value for a same flow: Water, TG
Different value for a same flow: Water, CU
Different value for a same flow: Water, IT
Different value for a same flow: Water, MD
Different value for a same flow: Water, CA
Different value for a same flow: Water, VI

Different value for a same flow: Water, BW
Different value for a same flow: Water, DM
Different value for a same flow: Water, UCTE
Different value for a same flow: Water, LU
Different value for a same flow: Water, US
Different value for a same flow: Water, MQ
Different value for a same flow: Water, NG
Different value for a same flow: Water, IQ
Different value for a same flow: Water, ES
Different value for a same flow: Water, DZ
Different value for a same flow: Water
Different value for a same flow: Water, CR
Different value for a same flow: Water, LT
Different value for a same flow: Water, GQ
Different value for a same flow: Water, GB
Different value for a same flow: Water, IAI Area, South America
Different value for a same flow: Water, LR
Different value for a same flow: Water, CO
Different value for a same flow: Water, GLO
Different value for a same flow: Water, EH
Different value for a same flow: Water, PH
Different value for a same flow: Water, BH
Different value for a same flow: W

Different value for a same flow: Water, TN
Different value for a same flow: Water, TL
Different value for a same flow: Water, RNA
Different value for a same flow: Water, CV
Different value for a same flow: Water, MK
Different value for a same flow: Water, NL
Different value for a same flow: Water, AF
Different value for a same flow: Water, BE
Different value for a same flow: Water, CM
Different value for a same flow: Water, HT
Different value for a same flow: Water, DE
Different value for a same flow: Water, HN
Different value for a same flow: Water, CF
Different value for a same flow: Water, MR
Different value for a same flow: Water, TT
Different value for a same flow: Water, RW
Different value for a same flow: Water (river water from technosphere, turbined)
Different value for a same flow: Water, RU
Different value for a same flow: Water, ET
Different value for a same flow: Water, GY
Different value for a same flow: Water, BI
Different value for a same flow: Water, RO
Different value

In [24]:
k_list = list(dict_openLCAvsWULCA)
dict_openLCAvsWULCA[k_list[0]],dict_openLCAvsWULCA[k_list[1]] =  res_compare_included_EF(olca_aware,  "Flow", WULCA_aware_new, "newname", to_print = "no" )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 1314


In [25]:
dict_openLCAvsWULCA[k_list[2]] = pd.DataFrame (
        {'oLCA_value': final_EF_combined(olca_aware,  "Flow", "Factor", WULCA_aware_new, "newname", "Factor", dict_openLCAvsWULCA[k_list[1]])[0] , 
         'WULCA_value': final_EF_combined(olca_aware,  "Flow", "Factor", WULCA_aware_new, "newname", "Factor", dict_openLCAvsWULCA[k_list[1]])[1] },
        index = dict_openLCAvsWULCA[k_list[1]]
    )

Different value for a same flow: Water, MG
Different value for a same flow: Water, RME
Different value for a same flow: Water, AD
Different value for a same flow: Water, DK
Different value for a same flow: Water, TO
Different value for a same flow: Water, AW
Different value for a same flow: Water, KM
Different value for a same flow: Water, RoW
Different value for a same flow: Water, NC
Different value for a same flow: Water, AT
Different value for a same flow: Water, SD
Different value for a same flow: Water, GR
Different value for a same flow: Water, CZ
Different value for a same flow: Water, BF
Different value for a same flow: Water, TJ
Different value for a same flow: Water, CN
Different value for a same flow: Water, GM
Different value for a same flow: Water, NA
Different value for a same flow: Water, CL
Different value for a same flow: Water, BB
Different value for a same flow: Water, IS
Different value for a same flow: Water, UA
Different value for a same flow: Water, ER
Different

Different value for a same flow: Water, AD
Different value for a same flow: Water, DK
Different value for a same flow: Water, TO
Different value for a same flow: Water, AW
Different value for a same flow: Water, KM
Different value for a same flow: Water, GL
Different value for a same flow: Water, RoW
Different value for a same flow: Water, NC
Different value for a same flow: Water, AT
Different value for a same flow: Water, SD
Different value for a same flow: Water, GR
Different value for a same flow: Water, CZ
Different value for a same flow: Water, BF
Different value for a same flow: Water, TJ
Different value for a same flow: Water, CN
Different value for a same flow: Water, GM
Different value for a same flow: Water, NA
Different value for a same flow: Water, CL
Different value for a same flow: Water, BB
Different value for a same flow: Water, IS
Different value for a same flow: Water, UA
Different value for a same flow: Water, ER
Different value for a same flow: Water, PY
Different 

Different value for a same flow: Water, VG
Different value for a same flow: Water, NI
Different value for a same flow: Water, IE
Different value for a same flow: Water, MX
Different value for a same flow: Water, JP
Different value for a same flow: Water, VE
Different value for a same flow: Water, GW
Different value for a same flow: Water, LC
Different value for a same flow: Water, RER
Different value for a same flow: Water, KR
Different value for a same flow: Water, KN
Different value for a same flow: Water, AG
Different value for a same flow: Water, TD
Different value for a same flow: Water, LA
Different value for a same flow: Water, BD
Different value for a same flow: Water, YE
Different value for a same flow: Water, MG
Different value for a same flow: Water, RME
Different value for a same flow: Water, AD
Different value for a same flow: Water, DK
Different value for a same flow: Water, TO
Different value for a same flow: Water, AW
Different value for a same flow: Water, KM
Different

Different value for a same flow: Water, EH
Different value for a same flow: Water, PH
Different value for a same flow: Water, BH
Different value for a same flow: Water, VU
Different value for a same flow: Water, EG
Different value for a same flow: Water, HU
Different value for a same flow: Water, CD
Different value for a same flow: Water, OCE
Different value for a same flow: Water, IAI Area, Gulf Cooperation Council
Different value for a same flow: Water, MT
Different value for a same flow: Water, SV
Different value for a same flow: Water, PT
Different value for a same flow: Water, TG
Different value for a same flow: Water, CU
Different value for a same flow: Water, IT
Different value for a same flow: Water, MD
Different value for a same flow: Water, CA
Different value for a same flow: Water, PM
Different value for a same flow: Water, TM
Different value for a same flow: Water, NP
Different value for a same flow: Water, AE
Different value for a same flow: Water, Europe without Switzerla

Different value for a same flow: Water, IQ
Different value for a same flow: Water, ES
Different value for a same flow: Water, DZ
Different value for a same flow: Water
Different value for a same flow: Water, CR
Different value for a same flow: Water, LT
Different value for a same flow: Water, GQ
Different value for a same flow: Water, GB
Different value for a same flow: Water, LR
Different value for a same flow: Water, CO
Different value for a same flow: Water, GLO
Different value for a same flow: Water, EH
Different value for a same flow: Water, PH
Different value for a same flow: Water, BH
Different value for a same flow: Water, VU
Different value for a same flow: Water, EG
Different value for a same flow: Water, HU
Different value for a same flow: Water, CD
Different value for a same flow: Water, OCE
Different value for a same flow: Water, MT
Different value for a same flow: Water, SV
Different value for a same flow: Water, PT
Different value for a same flow: Water, TG
Different val

In [26]:
for name in pairwise_name:
    new_df = globals()['dict_%s' % name][name + "_result_diffEFs"]
    try:
        new_df_corr = new_df.corr()
    except: 
        error
    print(new_df_corr)
    globals()['dict_%s' % name][name + "_result_corr"] = new_df_corr

            SP_value  olca_value
SP_value         1.0         1.0
olca_value       1.0         1.0
             SP_value  WULCA_value
SP_value     1.000000     0.889463
WULCA_value  0.889463     1.000000
             oLCA_value  WULCA_value
oLCA_value     1.000000     0.889463
WULCA_value    0.889463     1.000000


In [27]:
# saving each dict_ to a single worksheet, different rows , # saving to sub-folder "results/"

if not os.path.exists("results"):
    os.makedirs("results")

filename = "AWARE" + "_pairwise_comp_result.xlsx"
writer = pd.ExcelWriter("results/" + filename, engine='xlsxwriter')

for name in pairwise_name:
    sum_table = globals()['dict_%s' % name][name + "_commonEF_sumtable"]
    new_df = globals()['dict_%s' % name][name + "_result_diffEFs"]
    new_df_corr = globals()['dict_%s' % name][name + "_result_corr"]

    sum_table.to_excel(writer,sheet_name=name,startrow=0 , startcol=0)   # write first  (3rows)
    new_df_corr.to_excel(writer,sheet_name=name,startrow=5 , startcol=0) # write next   (3rows)
    new_df.to_excel(writer,sheet_name=name,startrow=10 , startcol=0)     # finally, write the resulting EFs 
     
writer.save()